# Generative Adversarial Networks

Generative Adversarial Networks (GANs) are an approach to generative modeling based on deep learning methods.

The standard problem settings for GANs are generation of photorealistic images and image-to-image translation tasks (translating photos of summer to winter, day to night etc.).

In this task you will familiarize yourself with both these problems while trying to create fake images of sneakers. 

As GANs still do have certain limitations about generating large images, the task is decomposed into two: first, use a simple GAN to generate a bunch of low resolution images from noise, then upscale them using another generative model.

This notebook is built around [PyTorch](https://pytorch.org/) and [Lightning](https://pytorchlightning.ai/).

In [4]:
!pip install pytorch_lightning

     |████████████████████████████████| 776 kB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 197 kB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 147.0 MB 3.6 MB/s eta 0:00:011   |███▏                            | 14.3 MB 2.7 MB/s eta 0:00:50
     |████████████████████████████████| 168 kB 6.5 MB/s eta 0:00:01
     |████████████████████████████████| 806 kB 5.9 MB/s eta 0:00:01
     |████████████████████████████████| 397 kB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 55 kB 5.0 MB/s eta 0:00:01
     |████████████████████████████████| 83 kB 4.7 MB/s eta 0:00:011
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
     |████████████████████████████████| 5.7 MB 4.0 MB/s eta 0:00:01
     |████████████████████████████████| 536 kB 5.9 MB/s eta 0:00:01


In [5]:
%%bash

if [ ! -d data/ ]; then
    curl -sO 'https://code.mipt.ru/courses-public/cv/storage/-/raw/tasks/sneaker-generation/data.zip'
    unzip -qo data.zip
fi

In [6]:
import os

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torchvision
from IPython.display import clear_output

ModuleNotFoundError: No module named 'cv2'

In [ ]:
BATCH_SIZE = 128
IMAGE_SIZE = (28, 28)
NOISE_DIM = 100
LOW_RES_SIZE = IMAGE_SIZE
HIGH_RES_SIZE = (112, 112)

In [ ]:
train_kwargs = {}
dataset_root = "./data"
images_dir = "images"
image_filenames = sorted(os.listdir(os.path.join(dataset_root, images_dir)))
len(image_filenames)

# I. Image Generation

Your first task is to solve a problem of generating photorealistic images out of noise (okay, this might sound optimistic).

Namely, you are required to **create fake images of sneakers of resolution 28x28 given a vector of noise sampled from standard normal distribution.**

Here is simple PyTorch `Dataset` class for loading and preprocessing

In [ ]:
class SneakersDataset(torch.utils.data.Dataset):
    def __init__(
        self,
        root_dir: str,
        images_dir="images",
        input_size=None,
        target_size=None,
    ):
        self.images_dir = os.path.join(root_dir, images_dir)
        self.input_size = input_size
        self.target_size = target_size
        files = os.listdir(self.images_dir)
        self.all_images = sorted([file for file in files if file.endswith(".jpg")])

    def __len__(self):
        return len(self.all_images)

    def __getitem__(self, idx):
        image_path = os.path.join(self.images_dir, self.all_images[idx])
        image_bgr = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
        if self.input_size is not None:
            input_rgb = cv2.resize(image_rgb, self.input_size)
            input_rgb = (input_rgb / 255).astype(np.float32)
        if self.target_size is not None:
            image_rgb = cv2.resize(image_rgb, self.target_size)
        image_rgb = (image_rgb / 255).astype(np.float32)

        # tanh values in [-1, 1]
        image_rgb = torch.from_numpy(image_rgb * 2 - 1)
        if self.input_size is not None:
            input_rgb = torch.from_numpy(input_rgb * 2 - 1)
            return input_rgb, image_rgb
        else:
            return image_rgb

Here we create `LightningDataModule` which will handle dataset loading in our case

In [ ]:
class SneakersGANDataModule(pl.LightningDataModule):
    def __init__(self, data_dir: str, batch_size, shuffle=True):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.shuffle = shuffle

    def setup(self, stage=None):
        self.dataset = SneakersDataset(self.data_dir, target_size=IMAGE_SIZE)

    def train_dataloader(self):
        return torch.utils.data.DataLoader(
            self.dataset,
            batch_size=self.batch_size,
            shuffle=self.shuffle,
        )

In [ ]:
def data2img(d: np.ndarray):
    return 0.5 * d + 0.5


def visualize_images(data, n_rows, n_cols):
    n_samples = n_rows * n_cols

    if len(data) != n_samples:
        sample_indices = np.random.choice(
            len(data),
            n_samples,
            replace=len(data) < n_samples,
        )
    else:
        sample_indices = np.arange(len(data)).astype(int)

    plt.figure(figsize=(int(2.5 * n_cols), int(2.5 * n_rows)))
    for i, sample_index in enumerate(sample_indices):
        plt.subplot(n_rows, n_cols, i + 1)
        plt.imshow(data2img(data[sample_index]))
        plt.axis("off")
    plt.show()

In [ ]:
dm = SneakersGANDataModule("data", batch_size=32)
dm.setup()
visualize_images(next(iter(dm.train_dataloader())), 4, 8)

## Fully connected GAN

### Generator (0.5 pts) [cross-check:0]

Our first step is to build a generator. You should use the layers in `torch.nn` (imported in the beginning as `nn`) to construct the model. Since we are using PyTorch and Lightning, you should create `nn.Module`. Use the default initializers for parameters.

Architecture:
 * Fully connected (`Linear` in PyTorch) with output size of 1024
 * ReLU
 * Fully connected with output size of 1024
 * ReLU
 * Fully connected with output size of 28 x 28 x 3
 * TanH (to restrict every element of the output to be in the range [-1,1])
 * Reshape into (28, 28, 3)

> You could perform reshaping inside of `forward` method

In [ ]:
class FCGenerator(nn.Module):
    def __init__(self, noise_dim: int, img_shape: tuple):
        super().__init__()
        self.img_shape = img_shape

        self.model = nn.Sequential(
            nn.Linear(noise_dim, 1024),
            nn.ReLU(inplace=True),
            # TODO: add other layers
            ...,
        )

    def forward(self, z):
        img = self.model(z)
        # keep batch size
        img = img.view(img.size(0), *self.img_shape)
        return img

In [ ]:
fc_gen = FCGenerator(NOISE_DIM, IMAGE_SIZE + (3,))
img_generated = fc_gen(torch.randn(32, NOISE_DIM))
assert img_generated.shape[1:] == IMAGE_SIZE + (3,)

### Discriminator (0.5 pts) [cross-check:1]

Now you are to build a discriminator. You should use the default initializers for parameters here as well.

Architecture:
 * Flatten
 * Fully connected with output size of 256
 * Leaky ReLU(0.01)
 * Fully connected with output size of 256
 * Leaky ReLU(0.01)
 * Fully connected with output size of 1
 * Sigmoid (to obtain logits as an output)

The output of the discriminator should thus have shape `[batch_size, 1]`, and contain real numbers corresponding to the probability that each of the `batch_size` inputs is a real image.

> You could perform flattening inside of `forward` method

In [ ]:
class FCDiscriminator(nn.Module):
    def __init__(self, img_shape: tuple):
        super().__init__()

        self.model = nn.Sequential(
            # TODO: add layers
            ...
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        return self.model(img_flat)

In [ ]:
fc_dis = FCDiscriminator(IMAGE_SIZE + (3,))
prob_dis = fc_dis(img_generated)
assert prob_dis.shape[1:] == (1,)

## GAN Loss (1.0 pts) [cross-check:2]

Compute the generator and discriminator loss. The generator loss is:
$$\ell_G  =  -\mathbb{E}_{z \sim p(z)}\left[\log D(G(z))\right]$$
and the discriminator loss is:
$$ \ell_D = -\mathbb{E}_{x \sim p_\text{data}}\left[\log D(x)\right] - \mathbb{E}_{z \sim p(z)}\left[\log \left(1-D(G(z))\right)\right]$$

Instead of computing the expectation, you may average over elements of the minibatch, so make sure to combine the loss by *averaging* instead of summing.

Note that these are negated from the equations presented earlier as we will be *minimizing* these losses.

There is a `LightningModule` with all necessary methods. You should fill all `TODO` comments.

> Don't forget to use `detach` method for discriminator training step to prevent backpropagation for generator compuational graph

In [ ]:
class FCGAN(pl.LightningModule):
    def __init__(
        self,
        width,
        height,
        channels,
        noise_dim=100,
        lr=0.0002,
        b1=0.5,
        b2=0.999,
    ):
        super().__init__()
        # make <arg> available as self.hparams.<arg>
        self.save_hyperparameters()

        # Important: This property activates manual optimization.
        self.automatic_optimization = False

        img_shape = (width, height, channels)
        self.generator = FCGenerator(
            noise_dim=self.hparams.noise_dim,
            img_shape=img_shape,
        )
        self.discriminator = FCDiscriminator(
            img_shape=img_shape,
        )

        self.validation_z = torch.randn(8, noise_dim)

    def forward(self, z):
        return self.generator(z)

    def gan_loss(self, y_hat, y):
        # TODO: implement GAN loss (hint: binary cross entropy)
        return ...

    def training_step(self, imgs, batch_idx):
        opt_g, opt_d = self.optimizers()

        z = torch.randn(imgs.size(0), self.hparams.noise_dim)
        # move to same device as imgs
        z = z.type_as(imgs)

        # optimize generator
        self.generated_imgs = self(z)

        # all fake, but we want to be real
        valid = torch.ones(imgs.size(0), 1)
        valid = valid.type_as(imgs)

        g_loss = self.gan_loss(self.discriminator(self(z)), valid)
        self.log("g_loss", g_loss, prog_bar=True)

        opt_g.zero_grad()
        self.manual_backward(g_loss)
        opt_g.step()

        # optimize discriminator
        valid = torch.ones(imgs.size(0), 1)
        valid = valid.type_as(imgs)
        # TODO: loss for `discriminator(imgs)` and `valid`
        real_loss = ...

        # TODO: zero vector for fake_loss computation
        fake = ...
        fake = fake.type_as(imgs)

        fake_loss = self.gan_loss(self.discriminator(self(z).detach()), fake)

        d_loss = (real_loss + fake_loss) / 2
        self.log("d_loss", d_loss, prog_bar=True)

        opt_d.zero_grad()
        self.manual_backward(d_loss)
        opt_d.step()

    def configure_optimizers(self):
        lr = self.hparams.lr
        b1 = self.hparams.b1
        b2 = self.hparams.b2

        opt_g = torch.optim.Adam(self.generator.parameters(), lr=lr, betas=(b1, b2))
        opt_d = torch.optim.Adam(self.discriminator.parameters(), lr=lr, betas=(b1, b2))
        return [opt_g, opt_d], []

    def on_epoch_end(self):
        # send results for validation_z to TensorBoard
        z = self.validation_z.type_as(self.generator.model[0].weight)
        # channels before pixels
        sample_imgs = self(z).permute(0, 3, 1, 2)
        grid = torchvision.utils.make_grid(sample_imgs)
        self.logger.experiment.add_image("generated_images", grid, self.current_epoch)

#### Training

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir lightning_logs

In [ ]:
train_kwargs["max_epochs"] = 100

In [ ]:
dm = SneakersGANDataModule("data", batch_size=BATCH_SIZE)
model = FCGAN(*IMAGE_SIZE, 3, NOISE_DIM)
trainer = pl.Trainer(**train_kwargs)
trainer.fit(model, dm)

In [ ]:
visualize_images(model(torch.randn(32, NOISE_DIM)).detach(), 4, 8)

> Hint: this architecture isn't great, so don't spend all your time to train it

## Deep Convolutional GANs
In the first part of the notebook, you have implemented an almost direct copy of the original GAN network from Ian Goodfellow. However, this network architecture allows no real spatial reasoning. It is unable to reason about things like "sharp edges" in general because it lacks any convolutional layers. Thus, in this section, you are to implement some of the ideas from [DCGAN](https://arxiv.org/abs/1511.06434), where both discriminator and generator are convolutional networks.

### Generator (1.0 pts) [cross-check:3]

Architecture:
 * Fully connected with output size of 128x7x7
 * Reshape into (128, 7, 7)
 * ReLU
 * UpSampling2D(2)
 * Conv2D: 3x3, filters=128, padding="same"
 * Batch Normalization 2D with momentum(0.8)
 * ReLU
 * UpSampling2D(2)
 * Conv2D: 3x3, filters=64, padding="same"
 * Batch Normalization 2D with momentum(0.8)
 * ReLU
 * Conv2D: 3x3, filters=3, padding="same"
 * TanH (to restrict every element of the output to be in the range [-1,1])

> There is no `padding="same"` in PyTorch, but you could use `padding=k//2` and `padding_mode="zeros"` for kernel size `k`

In [ ]:
class DCGenerator(nn.Module):
    def __init__(self, noise_dim: int, img_shape: tuple):
        super().__init__()
        self.img_shape = img_shape

        self.model = nn.Sequential(
            # TODO: add layers
            ...
        )

    def forward(self, z):
        img = self.model(z)
        # make channel axis last
        img = img.permute(0, 2, 3, 1)
        return img

In [ ]:
dc_gen = DCGenerator(NOISE_DIM, IMAGE_SIZE + (3,))
fake_imgs = dc_gen(torch.randn(10, NOISE_DIM))
assert fake_imgs.shape[1:] == IMAGE_SIZE + (3,)

### Discriminator (1.0 pts) [cross-check:4]

Architecture:
 * Conv2D: 3x3, filters=32, strides=2, padding="same"
 * Leaky ReLU(0.2)
 * Dropout(0.25)
 * Conv2D: 3x3, filters=64, strides=2, padding="same"
 * Zero Padding 2D: ((0, 1), (0, 1))
 * Batch Normalization 2D with momentum(0.8)
 * Leaky ReLU(0.2)
 * Dropout(0.25)
 * Conv2D: 3x3, filters=128, strides=2, padding="same"
 * Batch Normalization 2D with momentum(0.8)
 * Leaky ReLU(0.2)
 * Dropout(0.25)
 * Conv2D: 3x3, filters=256, strides=2, padding="same"
 * Batch Normalization 2D with momentum(0.8)
 * Leaky ReLU(0.2)
 * Dropout(0.25)
 * Flatten
 * Fully connected layer with output size 1
 * Sigmoid

In [ ]:
class DCDiscriminator(nn.Module):
    def __init__(self, img_shape: tuple):
        super().__init__()

        self.model = nn.Sequential(
            # TODO: add layers
            ...
        )

    def forward(self, img):
        # channels first
        img_chan = img.permute(0, 3, 1, 2)
        return self.model(img_chan)

In [ ]:
dc_dis = DCDiscriminator(img_shape=IMAGE_SIZE + (3,))
fake_proba = dc_dis(fake_imgs)
assert fake_proba.shape[1:] == (1,)

## Least Squares GAN loss (1.0 pts) [cross-check:5]
We'll now look at [Least Squares GAN loss](https://arxiv.org/abs/1611.04076), a newer, more stable alternative to the original GAN loss function. For this part, all you have to do is change the loss function and retrain the model. You'll implement equation (9) in the paper, with the generator loss:
$$\ell_G  =  \frac{1}{2}\mathbb{E}_{z \sim p(z)}\left[\left(D(G(z))-1\right)^2\right]$$
and the discriminator loss:
$$ \ell_D = \frac{1}{2}\mathbb{E}_{x \sim p_\text{data}}\left[\left(D(x)-1\right)^2\right] + \frac{1}{2}\mathbb{E}_{z \sim p(z)}\left[ \left(D(G(z))\right)^2\right]$$

In [ ]:
class DCGAN(pl.LightningModule):
    def __init__(
        self,
        width,
        height,
        channels,
        noise_dim=100,
        lr=0.0002,
        b1=0.5,
        b2=0.999,
    ):
        super().__init__()
        # make <arg> available as self.hparams.<arg>
        self.save_hyperparameters()

        # Important: This property activates manual optimization.
        self.automatic_optimization = False

        img_shape = (width, height, channels)
        self.generator = DCGenerator(
            noise_dim=self.hparams.noise_dim,
            img_shape=img_shape,
        )
        self.discriminator = DCDiscriminator(
            img_shape=img_shape,
        )

        self.validation_z = torch.randn(8, noise_dim)

    def forward(self, z):
        return self.generator(z)

    def ls_loss(self, y_hat, y):
        # TODO: implement least squares GAN loss
        return ...

    def training_step(self, imgs, batch_idx):
        opt_g, opt_d = self.optimizers()

        z = torch.randn(imgs.size(0), self.hparams.noise_dim)
        # move to same device as imgs
        z = z.type_as(imgs)

        # optimize generator
        # TODO: compute loss as before (FCGAN)
        ...

        self.log("g_loss", g_loss, prog_bar=True)

        opt_g.zero_grad()
        self.manual_backward(g_loss)
        opt_g.step()

        # optimize discriminator
        # TODO: compute loss as before (FCGAN)
        ...

        self.log("d_loss", d_loss, prog_bar=True)

        opt_d.zero_grad()
        self.manual_backward(d_loss)
        opt_d.step()

    def configure_optimizers(self):
        lr = self.hparams.lr
        b1 = self.hparams.b1
        b2 = self.hparams.b2

        opt_g = torch.optim.Adam(self.generator.parameters(), lr=lr, betas=(b1, b2))
        opt_d = torch.optim.Adam(self.discriminator.parameters(), lr=lr, betas=(b1, b2))
        return [opt_g, opt_d], []

    def on_epoch_end(self):
        z = self.validation_z.type_as(self.generator.model[0].weight)
        # channels before pixels
        sample_imgs = self(z).permute(0, 3, 1, 2)
        grid = torchvision.utils.make_grid(sample_imgs)
        self.logger.experiment.add_image("generated_images", grid, self.current_epoch)

### Training
Train generator and discriminator in a loop and draw results once every N iterations.

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir lightning_logs

In [ ]:
train_kwargs["max_epochs"] = 100

In [ ]:
dm = SneakersGANDataModule("data", batch_size=BATCH_SIZE)
model = DCGAN(*IMAGE_SIZE, 3, NOISE_DIM)
trainer = pl.Trainer(**train_kwargs)
trainer.fit(model, dm)

Prepare data for **III. GAN metrics. PRD score**:

In [ ]:
import tqdm


def predict(
    model: pl.LightningModule,
    data: torch.Tensor,
    batch_size: int,
    verbose: bool = True,
):
    output = []
    slicer = range(0, len(data), batch_size)
    if verbose:
        slicer = tqdm.tqdm(slicer)

    model.eval()
    use_gpu = torch.cuda.is_available()
    if use_gpu:
        model.cuda()
    with torch.no_grad():
        for i in slicer:
            x = data[i : i + batch_size]
            if use_gpu:
                x = x.cuda()
            y = model(x)
            if use_gpu:
                y = y.cpu()
            output.append(y)
    if use_gpu:
        model.cpu()

    output = torch.cat(output, dim=0)
    return output

In [ ]:
dataset = SneakersDataset("data", target_size=IMAGE_SIZE)
lr_data = torch.stack([dataset[i] for i in range(len(dataset))], dim=0)
dc_fake_data = predict(model, torch.randn(lr_data.size(0), NOISE_DIM), BATCH_SIZE)
print("LR data:", lr_data.size())
print("Fake LR data:", dc_fake_data.size())

In [ ]:
visualize_images(dc_fake_data, 4, 8)

# II. Super Resolution

In this part of the notebook you will train a generative model that solves an image-to-image problem, with "small images" as a source domain and "large images" being a target domain. 

To specify the task, you are to **scale small images of 28x28 pixels up to size of 112x112 pixels**.

In [ ]:
class SneakersSRDataModule(pl.LightningDataModule):
    def __init__(self, data_dir: str, batch_size, shuffle=True):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.shuffle = shuffle

    def setup(self, stage=None):
        self.dataset = SneakersDataset(
            self.data_dir,
            input_size=LOW_RES_SIZE,
            target_size=HIGH_RES_SIZE,
        )

    def train_dataloader(self):
        return torch.utils.data.DataLoader(
            self.dataset,
            batch_size=self.batch_size,
            shuffle=self.shuffle,
        )

In [ ]:
batch_size = 4
dm = SneakersSRDataModule("data", batch_size=batch_size)
dm.setup()
real_data_lr, real_data_hr = next(iter(dm.train_dataloader()))

assert real_data_lr.shape[1:] == LOW_RES_SIZE + (3,)
assert real_data_hr.shape[1:] == HIGH_RES_SIZE + (3,)

plt.figure(figsize=(8, 14))
for i in range(batch_size):
    plt.subplot(batch_size, 2, 2 * i + 1)
    plt.title("Low resolution")
    plt.imshow(data2img(real_data_lr[i]))
    plt.axis("off")

    plt.subplot(batch_size, 2, 2 * i + 2)
    plt.title("High resolution")
    plt.imshow(data2img(real_data_hr[i]))
    plt.axis("off")

plt.show()

In the second part of this task, you are to train an [SRGAN](https://arxiv.org/abs/1609.04802)-like model. For your convinience, some layers are already implemented. Now it's your turn -- fill the gaps so the model passes the asserts below.

### Generator (1.5 pts) [cross-check:6]

To build a SRGAN Generator, you will need a basic Residual Block(filters):

* Conv2D: 3x3, filters=filters, strides=1, padding="same"
* ReLU
* Batch Normalization 2D with momentum(0.8)
* Conv2D: 3x3, filters=filters, strides=1, padding="same"
* Batch Normalization 2D with momentum(0.8)
* Sum up outputs with inputs


In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, filters: int):
        super().__init__()
        self.model = nn.Sequential(
            # TODO: add layers
            ...
        )

    def forward(self, z):
        return self.model(z) + z

Upsampling Block(filters):

* UpSampling2D(2)
* Conv2D: 3x3, filters=filters, strides=1, padding="same"
* ReLU


In [ ]:
class UpsamplingBlock(nn.Module):
    def __init__(self, filters: int, input_filters: int = None):
        super().__init__()
        self.model = nn.Sequential(
            # TODO: add layers
            ...
        )

    def forward(self, z):
        return self.model(z)

Now, using these basic building blocks, one is able to define a SRGAN generator:

In [ ]:
class SRGenerator(nn.Module):
    def __init__(self):
        super().__init__()

        self.init_conv = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=9, padding=9 // 2, padding_mode="zeros"),
            nn.ReLU(inplace=True),
        )
        self.residual_chain = nn.Sequential(
            *[ResidualBlock(64) for _ in range(16)],
            nn.Conv2d(64, 64, kernel_size=3, padding=3 // 2, padding_mode="zeros"),
            nn.BatchNorm2d(64, momentum=0.8)
        )
        self.upsample_conv = nn.Sequential(
            UpsamplingBlock(256, input_filters=64),
            UpsamplingBlock(256),
            nn.Conv2d(256, 3, kernel_size=9, padding=9 // 2, padding_mode="zeros"),
            nn.Tanh(),
        )

    def forward(self, z):
        x = z.permute(0, 3, 1, 2)
        conv = self.init_conv(x)
        x = self.residual_chain(conv) + conv
        img = self.upsample_conv(x)
        # make channel axis last
        img = img.permute(0, 2, 3, 1)
        return img

In [ ]:
real_data_lr, real_data_hr = next(iter(dm.train_dataloader()))
fake_hr = SRGenerator()(real_data_lr)
assert fake_hr.shape == real_data_hr.shape

### Discriminator (1.5 pts) [cross-check:7]

First, define a Discriminator Block(filters, strides):

* Conv2D: 3x3, filters=filters, strides=strides, padding="same"
* Leaky ReLU(0.2)
* (optional) Batch Normalization 2D with momentum(0.8) 

In [ ]:
class DBlock(nn.Module):
    def __init__(
        self,
        input_filters: int,
        filters: int,
        strides: int,
        batch_norm: bool = False,
    ):
        super().__init__()
        layers = [
            # TODO: add layers (except BatchNorm2d)
            ...
        ]
        if batch_norm:
            layers.append(nn.BatchNorm2d(filters, momentum=0.8))
        self.model = nn.Sequential(*layers)

    def forward(self, z):
        return self.model(z)

Now, use this block to build up SRGAN discriminator:

* DBlock(filters=64, strides=1) with Batch Normalization
* DBlock(filters=64, strides=2)
* DBlock(filters=128, strides=1)
* DBlock(filters=128, strides=2)
* DBlock(filters=256, strides=1)
* DBlock(filters=256, strides=2)
* DBlock(filters=512, strides=1)
* DBlock(filters=512, strides=2)
* Flatten
* Fully connected with output size of 1024
* Leaky ReLU(0.2)
* Fully connected with output size of 1
* Sigmoid

In [ ]:
class SRDiscriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            # TODO: add layers
            ...
        )

    def forward(self, z):
        x = z.permute(0, 3, 1, 2)
        return self.model(x)

In [ ]:
fake_probas = SRDiscriminator()(fake_hr)
assert fake_probas.shape[1:] == (1,)

Typically, SRGAN is trained with additional loss on features from pretrained VGG-19. However, you task will be a bit simplier: use **mean squared error** between real and fake data instead.

In [ ]:
class SRGAN(pl.LightningModule):
    def __init__(self, validation_lr):
        super().__init__()

        # Important: This property activates manual optimization.
        self.automatic_optimization = False

        self.generator = SRGenerator()
        self.discriminator = SRDiscriminator()

        self.validation_lr = validation_lr

    def forward(self, z):
        return self.generator(z)

    def gan_loss(self, y_hat, y):
        # TODO: implement loss
        return ...

    def ls_loss(self, y_hat, y):
        # TODO: implement loss
        return ...

    def mse_loss(self, sr_imgs, hr_imgs):
        # TODO: implement MSE loss
        return ...

    def training_step(self, imgs, batch_idx):
        opt_g, opt_d = self.optimizers()
        lr_imgs, hr_imgs = imgs

        # optimize generator
        sr_imgs = self(lr_imgs)

        # all fake, but we want to be real
        valid = torch.ones(lr_imgs.size(0), 1)
        valid = valid.type_as(lr_imgs)

        gan_loss = self.gan_loss(self.discriminator(sr_imgs), valid)
        mse_loss = self.mse_loss(sr_imgs, hr_imgs)

        g_loss = 1 * mse_loss + 0.5 * gan_loss
        self.log("g_loss", g_loss, prog_bar=True)
        self.log("gan_loss", gan_loss, prog_bar=True)
        self.log("mse_loss", mse_loss, prog_bar=True)

        opt_g.zero_grad()
        self.manual_backward(g_loss)
        opt_g.step()

        # optimize discriminator
        valid = torch.ones(hr_imgs.size(0), 1)
        valid = valid.type_as(hr_imgs)
        real_loss = self.ls_loss(self.discriminator(hr_imgs), valid)

        fake = torch.zeros(lr_imgs.size(0), 1)
        fake = fake.type_as(lr_imgs)
        fake_loss = self.ls_loss(self.discriminator(self(lr_imgs).detach()), fake)

        d_loss = (real_loss + fake_loss) / 2
        self.log("d_loss", d_loss, prog_bar=True)

        opt_d.zero_grad()
        self.manual_backward(d_loss)
        opt_d.step()

    def configure_optimizers(self):
        opt_g = torch.optim.Adam(
            self.generator.parameters(),
            lr=2e-4,
            betas=(0.5, 0.999),
        )
        opt_d = torch.optim.Adam(
            self.discriminator.parameters(),
            lr=1e-3,
            betas=(0.5, 0.999),
        )
        return [opt_g, opt_d], []

    def on_epoch_end(self):
        z = self.validation_lr.type_as(self.generator.init_conv[0].weight)
        # channels before pixels
        sample_imgs = self(z).permute(0, 3, 1, 2)
        grid = torchvision.utils.make_grid(sample_imgs)
        self.logger.experiment.add_image("generated_images", grid, self.current_epoch)

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir lightning_logs

### Training

In [ ]:
train_kwargs["max_epochs"] = 9

In [ ]:
dm = SneakersSRDataModule("data", batch_size=16)
dm.setup()
model = SRGAN(validation_lr=next(iter(dm.train_dataloader()))[0][:4])
trainer = pl.Trainer(**train_kwargs)
trainer.fit(model, dm)

Prepare data for **III. GAN metrics**:

In [ ]:
dm = SneakersSRDataModule("data", batch_size=BATCH_SIZE)
dm.setup()
hr_data = []
for lr, hr in dm.train_dataloader():
    hr_data.append(hr)
hr_data = torch.cat(hr_data, dim=0)

batch_size = 16
sr_fake_data_from_real_lr = predict(model, lr_data, batch_size)
sr_fake_data_from_fake_lr = predict(model, dc_fake_data, batch_size)
print("HR data:", hr_data.size())
print("SR from real LR:", sr_fake_data_from_real_lr.size())
print("SR from fake LR:", sr_fake_data_from_fake_lr.size())

In [ ]:
visualize_images(sr_fake_data_from_real_lr, 4, 8)

In [ ]:
visualize_images(sr_fake_data_from_fake_lr, 4, 8)

# III. GAN metrics

There exists a few metrics used to measure GAN performance. Some of them are based on comparing real samples against generated ones, while the other rely on additional pretrained models that are applied to both real and generated data in order to accumulate high-level statistics. In this task, you are going to use two metrics representing these two approaches -- namely, [Precision-Recall Density](https://arxiv.org/pdf/1806.00035) and [Fréchet Inception Distance](https://arxiv.org/pdf/1706.08500).

##  Precision-Recall Density (PRD score) (1.0 pts) [cross-check:8]

Your first task is to implement [Precision-Recall Density](https://arxiv.org/pdf/1806.00035.pdf) score.

In [ ]:
from sklearn.cluster import KMeans, MiniBatchKMeans


def bin_counts(real_data, fake_data, n_bins=25):
    real_data = real_data.reshape(len(real_data), -1)
    fake_data = fake_data.reshape(len(fake_data), -1)

    data = np.vstack([real_data, fake_data])

    kmeans = MiniBatchKMeans(n_clusters=n_bins, n_init=10).fit(data)

    real_labels = kmeans.labels_[: len(real_data)]
    fake_labels = kmeans.labels_[len(real_data) :]

    real_density, _ = np.histogram(
        real_labels,
        bins=n_bins,
        range=[0, n_bins],
        density=True,
    )
    # TODO: same for fake_labels
    fake_density, _ = ...

    return real_density, fake_density


def sample_bin_counts(real_data, fake_data, n_bins=25, repeat_number=10, verbose=True):
    real_densities = []
    fake_densities = []
    counter = range(repeat_number)
    if verbose:
        counter = tqdm.tqdm(counter)
    for _ in counter:
        real, fake = bin_counts(real_data, fake_data, n_bins=n_bins)
        real_densities.append(real)
        fake_densities.append(fake)
    return np.array(real_densities).mean(axis=0), np.array(fake_densities).mean(axis=0)

In [ ]:
import math

from sklearn.metrics import auc


def calculate_alpha_beta(real_density, fake_density, n_thetas=1000):
    assert real_density.shape == fake_density.shape

    alpha = []
    beta = []

    thetas = np.linspace(1e-6, np.pi / 2 - 1e-6, num=n_thetas)
    for theta in thetas:
        tan = math.tan(theta)
        # TODO: implement paper formula
        alpha.append(...)
        beta.append(...)

    return alpha, beta


def calculate_prd_score(real_data, fake_data):
    # Calculate bin counts from real and generated data multiple times
    # TODO
    real_density, fake_density = ...

    plt.bar(
        range(len(real_density)),
        real_density,
        width=1,
        color="g",
        alpha=0.5,
        label="Real density",
    )
    plt.bar(
        range(len(fake_density)),
        fake_density,
        width=1,
        color="r",
        alpha=0.5,
        label="Fake density",
    )
    plt.legend()
    plt.show()

    # Calculate alpha and beta
    # TODO
    alpha, beta = ...

    # Calculate area under curve (AUC) for alpha and beta
    # TODO
    score = ...

    return score, alpha, beta

Calculate PRD score for DCGAN (task I). You should pass `lr_data` and `dc_fake_data` to scoring function.

In [ ]:
score, _, _ = calculate_prd_score(
    lr_data,
    dc_fake_data,
)
print("Score:", score)

Now use PRD score to compare high resolution data generated from real low resolution data (`sr_fake_data_from_real_lr`) and fake resolution data (`sr_fake_data_from_fake_lr`):

In [ ]:
print("Generated from real LR")
score_real, alpha_real, beta_real = calculate_prd_score(
    hr_data,
    sr_fake_data_from_real_lr,
)
print("Score:", score_real, end="\n\n")

print("Generated from fake LR")
score_fake, alpha_fake, beta_fake = calculate_prd_score(
    hr_data,
    sr_fake_data_from_fake_lr,
)
print("Score:", score_fake, end="\n\n")

fig, ax = plt.subplots(figsize=(8, 8))
ax.tick_params(axis="both", which="major", labelsize=8)
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.plot(alpha_real, beta_real, color="g", label="Generated from real LR")
plt.plot(alpha_fake, beta_fake, color="r", label="Generated from fake LR")
plt.legend()
plt.show()

## Fréchet Inception Distance (FID score) (1.0 pts) [cross-check:9]

[Frechet Inception Distance](https://arxiv.org/pdf/1706.08500) is an improved version of [Inception score](https://arxiv.org/abs/1606.03498), that additionally calculates the statistics of real data and compares it to the statistics of generated data. It is probably the most widely-used option for evaluating GANs, and relies on features extracted with [InceptionV3](https://arxiv.org/abs/1512.00567) pretrained on ImageNet. These features assumed to come from a multivariate Gaussian distribution, so Fréchet distance between two multivariate Gaussians can be calculated:

$$\text{FID} = ||\mu_r - \mu_g||^2 + \text{Tr} (\Sigma_r + \Sigma_g - 2 (\Sigma_r \Sigma_g)^{1/2}),$$

where $X_r \sim \mathcal{N} (\mu_r, \Sigma_r)$ and $X_g \sim \mathcal{N} (\mu_g, \Sigma_g)$ are the 2048-dimensional activations of the Inception-v3 pool3 layer for real and generated samples respectively.

First, create InceptionV3 ([from torch repository](https://pytorch.org/hub/pytorch_vision_inception_v3/)) model. As you will be using it for feature extraction only, we should remove last fully connected layer.

In [ ]:
class InceptionHeadless(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.upsample = nn.Upsample(scale_factor=2)
        self.model = torch.hub.load("pytorch/vision", "inception_v3", pretrained=True)
        # remove last fc layer
        self.model.fc = nn.Identity()

    def forward(self, z):
        x = z.permute(0, 3, 1, 2)
        x = self.upsample(x)
        return self.model(x)

In [ ]:
inception = InceptionHeadless()
inception.eval()

with torch.no_grad():
    assert inception(hr_data[:32]).shape[1:] == (2048,)

In [ ]:
def calculate_activations(data, batch_size=32, verbose=False):
    # Calculate activations of Pool3 layer of InceptionV3
    if verbose:
        print("Calculating activations...")
    activations = predict(inception, data, batch_size=32)
    return activations


def calculate_activation_statistics(activations):
    # Calculate mean and covariance of activations. Mind the dimensions!
    # TODO
    mu = ...
    t = activations - mu
    # TODO
    sigma = ...
    return mu, sigma

In [ ]:
real_activations = calculate_activations(hr_data, verbose=True)
real_mu, real_sigma = calculate_activation_statistics(real_activations)

assert real_mu.shape == (2048,)
assert real_sigma.shape == (2048, 2048)

In [ ]:
import scipy


def calculate_frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
    assert mu1.shape == mu2.shape
    assert sigma1.shape == sigma2.shape

    sigma1_sigma2 = scipy.linalg.sqrtm(np.dot(sigma1, sigma2))

    # Numerical error might give slight imaginary component
    if np.iscomplexobj(sigma1_sigma2):
        sigma1_sigma2 = sigma1_sigma2.real

    # Product might be almost singular
    if not np.isfinite(sigma1_sigma2).all():
        offset = np.eye(sigma1.shape[0]) * eps
        sigma1_sigma2 = scipy.linalg.sqrtm(np.dot(sigma1 + offset, sigma2 + offset))

    diff = mu1 - mu2

    # use diff, sigma1, sigma2 to calculate FID according to the formula above
    # TODO: implement score from paper
    return ...


def calculate_fid_score(real_data, fake_data, verbose=False):
    # Run inception on real and fake data to obtain activations
    # TODO
    real_activations = ...
    fake_activations = ...

    # Calculate mu and sigma for both real and fake activations
    # TODO
    real_mu, real_sigma = ...
    fake_mu, fake_sigma = ...

    # Calculate Frechet distance
    return calculate_frechet_distance(
        real_mu,
        real_sigma,
        fake_mu,
        fake_sigma,
    )

Calculate FID score between `hr_data` and `sr_fake_data_from_real_hr`:

In [ ]:
score = calculate_fid_score(
    hr_data,
    sr_fake_data_from_real_lr,
)
print("Score:", score)

Putting it all together: calculate FID score between `hr_data` and `sr_fake_data_from_fake_hr`:

In [ ]:
score = calculate_fid_score(
    hr_data,
    sr_fake_data_from_fake_lr,
)
print("Score:", score)